[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/geoai/blob/main/docs/examples/dataviz/vector_viz.ipynb)
[![image](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/opengeos/geoai&urlpath=lab/tree/geoai/docs/examples/dataviz/vector_viz.ipynb&branch=main)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/opengeos/geoai/blob/main/docs/examples/dataviz/vector_viz.ipynb)

# Visualizing Vector Data with Leafmap

## Installation

Uncomment the following line to install [leafmap](https://leafmap.org) if needed.

In [ ]:
# %pip install "leafmap[vector]"

## Import libraries

In [ ]:
import leafmap

## Visualize vector data

You can visualize vector data using the `add_vector` function. It supports common vector data formats, including GeoJSON, Shapefile, GeoPackage, and any other formats supported by [geopandas](https://geopandas.org).

In [ ]:
m = leafmap.Map(center=[0, 0], zoom=2)
data = "https://opengeos.org/data/vector/cables.geojson"
m.add_vector(data, layer_name="Cable lines", info_mode="on_hover")
m

![](https://i.imgur.com/NIcnLWs.png)

You can style the vector with custom style callback functions. 

In [ ]:
m = leafmap.Map(center=[20, 0], zoom=2)
m.add_basemap("CartoDB.DarkMatter")
data = "https://opengeos.org/data/vector/cables.geojson"
callback = lambda feat: {"color": feat["properties"]["color"], "weight": 1}
m.add_vector(data, layer_name="Cable lines", style_callback=callback)
m

![](https://i.imgur.com/mQnV53U.png)

## Choropleth map

You can create a choropleth map using the `add_data` function. It supports GeoJSON, Shapefile, GeoPackage, and any other formats supported by [geopandas](https://geopandas.org). 


In [ ]:
m = leafmap.Map()
data = "https://raw.githubusercontent.com/opengeos/leafmap/master/docs/examples/data/countries.geojson"
m.add_data(
    data, column="POP_EST", scheme="Quantiles", cmap="Blues", legend_title="Population"
)
m

![](https://i.imgur.com/4FV6f72.png)

In [ ]:
m = leafmap.Map()
m.add_data(
    data,
    column="POP_EST",
    scheme="EqualInterval",
    cmap="Blues",
    legend_title="Population",
)
m

![](https://i.imgur.com/KEY6zEj.png)

## GeoParquet

Visualize GeoParquet data with leafmap and lonboard.

In [ ]:
url = "https://opengeos.org/data/duckdb/cities.parquet"
gdf = leafmap.read_parquet(url, return_type="gdf", src_crs="EPSG:4326")
gdf.head()

Visualize point data.

In [ ]:
leafmap.view_vector(
    gdf,
    get_radius=20000,
    get_fill_color="blue",
    zoom_to_layer=False,
    map_args={"center": (40, -100), "zoom": 3, "height": 500},
)

![](https://i.imgur.com/BBsLjvx.png)

Visualizing polygon data.

In [ ]:
url = "https://data.source.coop/giswqs/nwi/wetlands/DC_Wetlands.parquet"
gdf = leafmap.read_parquet(
    url, return_type="gdf", src_crs="EPSG:5070", dst_crs="EPSG:4326"
)
gdf.head()

In [ ]:
leafmap.view_vector(gdf, get_fill_color=[0, 0, 255, 128])

![vector](https://i.imgur.com/HRtpiVd.png)

Alternatively, you can specify a color map to visualize the data.

In [ ]:
color_map = {
    "Freshwater Forested/Shrub Wetland": (0, 136, 55),
    "Freshwater Emergent Wetland": (127, 195, 28),
    "Freshwater Pond": (104, 140, 192),
    "Estuarine and Marine Wetland": (102, 194, 165),
    "Riverine": (1, 144, 191),
    "Lake": (19, 0, 124),
    "Estuarine and Marine Deepwater": (0, 124, 136),
    "Other": (178, 134, 86),
}

In [ ]:
leafmap.view_vector(gdf, color_column="WETLAND_TYPE", color_map=color_map, opacity=0.5)

![vector-color](https://i.imgur.com/Ejh8hK6.png)

Display a legend for the data.

In [ ]:
leafmap.Legend(title="Wetland Type", legend_dict=color_map)

![legend](https://i.imgur.com/fxzHHFN.png)

## PMTiles

[PMTiles](https://github.com/protomaps/PMTiles) is a single-file archive format for tiled data. A PMTiles archive can be hosted on a commodity storage platform such as S3, and enables low-cost, zero-maintenance map applications that are "serverless" - free of a custom tile backend or third party provider.

## Remote PMTiles

Leafmap can visualize PMTiles hosted locally or remotely.

### Overture data

In [ ]:
import leafmap.foliumap as leafmap

Check the metadata of the PMTiles.

In [ ]:
url = "https://storage.googleapis.com/ahp-research/overture/pmtiles/overture.pmtiles"
metadata = leafmap.pmtiles_metadata(url)
print(f"layer names: {metadata['layer_names']}")
print(f"bounds: {metadata['bounds']}")

Visualize the PMTiles.

In [ ]:
m = leafmap.Map()
m.add_basemap("CartoDB.DarkMatter")

style = {
    "version": 8,
    "sources": {
        "example_source": {
            "type": "vector",
            "url": "pmtiles://" + url,
            "attribution": "PMTiles",
        }
    },
    "layers": [
        {
            "id": "admins",
            "source": "example_source",
            "source-layer": "admins",
            "type": "fill",
            "paint": {"fill-color": "#BDD3C7", "fill-opacity": 0.1},
        },
        {
            "id": "buildings",
            "source": "example_source",
            "source-layer": "buildings",
            "type": "fill",
            "paint": {"fill-color": "#FFFFB3", "fill-opacity": 0.5},
        },
        {
            "id": "places",
            "source": "example_source",
            "source-layer": "places",
            "type": "fill",
            "paint": {"fill-color": "#BEBADA", "fill-opacity": 0.5},
        },
        {
            "id": "roads",
            "source": "example_source",
            "source-layer": "roads",
            "type": "line",
            "paint": {"line-color": "#FB8072"},
        },
    ],
}

m.add_pmtiles(
    url, name="PMTiles", style=style, overlay=True, show=True, zoom_to_layer=True
)

legend_dict = {
    "admins": "BDD3C7",
    "buildings": "FFFFB3",
    "places": "BEBADA",
    "roads": "FB8072",
}

m.add_legend(legend_dict=legend_dict)
m

![](https://i.imgur.com/fjWMOu3.png)

### Source Cooperative

Visualize the [Google-Microsoft Open Buildings data](https://beta.source.coop/repositories/vida/google-microsoft-open-buildings/description) hosted on Source Cooperative.

Check the metadata of the PMTiles.

In [ ]:
url = "https://data.source.coop/vida/google-microsoft-open-buildings/pmtiles/go_ms_building_footprints.pmtiles"
metadata = leafmap.pmtiles_metadata(url)
print(f"layer names: {metadata['layer_names']}")
print(f"bounds: {metadata['bounds']}")

Visualize the PMTiles.

In [ ]:
m = leafmap.Map(center=[20, 0], zoom=2)
m.add_basemap("CartoDB.DarkMatter")
m.add_basemap("Esri.WorldImagery", show=False)

style = {
    "version": 8,
    "sources": {
        "example_source": {
            "type": "vector",
            "url": "pmtiles://" + url,
            "attribution": "PMTiles",
        }
    },
    "layers": [
        {
            "id": "buildings",
            "source": "example_source",
            "source-layer": "building_footprints",
            "type": "fill",
            "paint": {"fill-color": "#3388ff", "fill-opacity": 0.5},
        },
    ],
}

m.add_pmtiles(
    url, name="Buildings", style=style, overlay=True, show=True, zoom_to_layer=False
)

m

![](https://i.imgur.com/kT6ng6k.png)

## Local PMTiles

tippecanoe is required to convert vector data to pmtiles. Install it with `conda install -c conda-forge tippecanoe`.

Download [building footprints](https://github.com/opengeos/open-data/blob/main/datasets/libya/Derna_buildings.geojson) of Derna, Libya.

In [ ]:
url = "https://raw.githubusercontent.com/opengeos/open-data/main/datasets/libya/Derna_buildings.geojson"
leafmap.download_file(url, "buildings.geojson")

Convert vector to PMTiles.

In [ ]:
pmtiles = "buildings.pmtiles"
leafmap.geojson_to_pmtiles(
    "buildings.geojson", pmtiles, layer_name="buildings", overwrite=True, quiet=True
)

Start a HTTP Sever

In [ ]:
leafmap.start_server(port=8000)

In [ ]:
url = f"http://127.0.0.1:8000/{pmtiles}"
leafmap.pmtiles_metadata(url)

Display the PMTiles on the map.

In [ ]:
m = leafmap.Map()

style = {
    "version": 8,
    "sources": {
        "example_source": {
            "type": "vector",
            "url": "pmtiles://" + url,
            "attribution": "PMTiles",
        }
    },
    "layers": [
        {
            "id": "buildings",
            "source": "example_source",
            "source-layer": "buildings",
            "type": "fill",
            "paint": {"fill-color": "#3388ff", "fill-opacity": 0.5},
        },
    ],
}

m.add_pmtiles(url, name="Buildings", show=True, zoom_to_layer=True, style=style)
m

![](https://i.imgur.com/lhGri31.png)